In [116]:
import tweepy
import time
import pandas as pd
import sys
from tweepy import OAuthHandler
import json

In [117]:
with open('C:\\Bala\\DataScience\\Twitter\\config.json') as t_file:
    json = json.load(t_file)
    
## TODO : ReformedBroker

In [118]:
consumer_key = json['twitter']['consumer_key']
consumer_secret = json['twitter']['consumer_secret']
access_token_key=json['twitter']['access_token_key']
access_token_secret=json['twitter']['access_token_secret']

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token,access_secret)

api = tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True,compression=True)

if(api.verify_credentials):
    print('We successfully logged in')
    
json['accounts']

We successfully logged in


['ReformedBroker']

In [119]:
def lookup_user_list(user_id_list,api):
    all_users = []
    users_count = len(user_id_list)
    while True:
        try:
            for i in range(users_count%100 + 1):
                print(i)
                all_users.extend(api.lookup_users(user_ids=user_id_list[i*100 : min((i+1)*100,users_count)]))        
        except tweepy.TweepError(e):
            print('Something went wrong... ')
            time.sleep(60*15)
        return all_users

In [ ]:
for name in json['accounts']:
    print('Fetching followers for ' + name)
    ids = []
    i = 0
    backoff_counter = 1
    while True:
        try:            
            for page in tweepy.Cursor(api.followers_ids,screen_name=name,count=5000).pages(): 
                if(i >= 14):
                    time.sleep(60*15) 
                ids.extend(page)
                print(i)
                i+=1
                time.sleep(60)
            break
        except tweepy.TweepError as e:
            print(e.reason)
            time.sleep(60*backoff_counter)
            backoff_counter += 1
            continue
            
    print(len(ids))  
    results = lookup_user_list(ids,api)
    all_users = [{'id': user.id,
             'Name': user.name,
             'Statuses Count': user.statuses_count,
             'Friends Count': user.friends_count,
             'Screen Name': user.screen_name,
             'Followers Count': user.followers_count,
             'Location': user.location,
             'Language': user.lang,
             'Created at': user.created_at,
             'Time zone': user.time_zone,
             'Geo enable': user.geo_enabled,
             'Description': user.description}
             for user in results]
    df = pd.DataFrame(all_users)
    df.to_csv('C:\\Bala\\DataScience\\Twitter\\Followers-'+name+'.csv', index=False, encoding='utf-8')


Fetching followers for ReformedBroker
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Failed to send request: ('Connection aborted.', OSError("(10054, 'WSAECONNRESET')",))
15
